In [1]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
#tf.enable_eager_execution()
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pickle
import numpy as np

In [2]:
with open('encodedStates.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [3]:
def df_to_np(dataframe,shuffle=True,batch_size=32,target="HHVEHCNT"):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target)
    return dataframe.values, labels.values

In [6]:
class TransferLearn:
    
    def __init__(self, fromState, toState, layers_to_transfer, verbose):
        self.data = data
        self.fromState = fromState
        self.toState = toState
        self.layers_to_transfer = layers_to_transfer
        self.verbose = verbose
        
        
        self.train_X_from, self.train_y_from = None, None
        self.train_X_to, self.train_y_to = None, None
        self.fromModelWeights = None
    
    def initFromStateData(self, sample_size = None):
        if sample_size:
            self.train_X_from, self.train_y_from = df_to_np(data[self.fromState].sample(sample_size))
            return self.train_X_from, self.train_y_from
        else:
            self.train_X_from, self.train_y_from = df_to_np(data[self.fromState])
            return self.train_X_from, self.train_y_from
    
    def initToStateData(self, sample_size = None):
        if sample_size:
            self.train_X_to, self.train_y_to = df_to_np(data[self.toState].sample(sample_size))
            return self.train_X_to, self.train_y_to
        else:
            self.train_X_to, self.train_y_to = df_to_np(data[self.toState])
            return self.train_X_to, self.train_y_to
    
    #Get the weights from model built on toStateData
    def getFromModelWeights(self, batch_size=100, validation_split=0.2, epochs=20):
        train_X_from = self.train_X_from.copy()
        train_y_from = self.train_y_from.copy()
        model = tf.keras.Sequential([
          tf.keras.layers.Dense(128,activation="relu"),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(64,activation="relu"),
          tf.keras.layers.Dense(4,activation="softmax")
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(x=self.train_X_from, y = self.train_y_from,epochs=epochs, batch_size = batch_size, validation_split=validation_split, shuffle=True, verbose=self.verbose)
        
        weights = []
        for i in range(0,self.layers_to_transfer):
            weights.append(model.layers[i].get_weights())
        self.fromModelWeights = weights
        return self.fromModelWeights
    
    def transfer(self, trainable=True, batch_size=100, validation_split=0.2, epochs=20):
        train_X_to = self.train_X_to.copy()
        train_y_to = self.train_y_to.copy()
        weights = self.fromModelWeights if self.fromModelWeights else self.getFromModelWeights()
        print("*****weights obtained from fromStateData*****")
        
        print("Transferring")
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(128,activation="relu", weights=weights[0], trainable=trainable))
        for i in range(1,self.layers_to_transfer):
            model.add(tf.keras.layers.Dense(128, weights=weights[i], trainable=trainable))
        for i in range(4-self.layers_to_transfer,4):
            model.add(tf.keras.layers.Dense(128))
        model.add(tf.keras.layers.Dense(64,activation="relu"))
        model.add(tf.keras.layers.Dense(4,activation="softmax"))

        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        history = model.fit(x=self.train_X_to, y = self.train_y_to, epochs=epochs, batch_size = batch_size, validation_split=validation_split, shuffle=True,verbose=self.verbose)
        val_acc = history.history["val_accuracy"][-1]
        print("*****Transferring done*****", "Trainable:", trainable)
        return model, val_acc
    
    
    def benchmark(self, batch_size=100, validation_split=0.2, epochs=20):
        train_X_to = self.train_X_to.copy()
        train_y_to = self.train_y_to.copy()
        print("*****Training Benchmark Model*****")
        model = tf.keras.Sequential([
          tf.keras.layers.Dense(128,activation="relu"),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(64,activation="relu"),
          tf.keras.layers.Dense(4,activation="softmax")
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        history = model.fit(x=self.train_X_to, y = self.train_y_to,epochs=epochs, batch_size = batch_size, validation_split=validation_split, shuffle=True,verbose=self.verbose)
        val_acc = history.history["val_accuracy"][-1]
        print("*****Training Benchmark Model Done*****")
        return model, val_acc
    
    def compare(self):
        benchmark_model, benchmark_acc = self.benchmark()
        frozenTransfer_model, frozenTransfer_acc = self.transfer(trainable=False)
        unfrozenTransfer_model, unfrozenTransfer_acc = self.transfer(trainable=True)
        
        return (benchmark_acc-frozenTransfer_acc, benchmark_acc-unfrozenTransfer_acc)
        
    

In [12]:
MA_to_NY = TransferLearn(fromState = "MA", toState = "NY", layers_to_transfer=2, verbose = 1)
MA_to_NY.initFromStateData()
MA_to_NY.initToStateData(500)
compareMAandNY = MA_to_NY.compare()

*****Training Benchmark Model*****
Train on 400 samples, validate on 100 samples
Epoch 1/15
400/400 [==============================] - 1s 2ms/sample - loss: 1.2770 - accuracy: 0.3425 - val_loss: 1.1513 - val_accuracy: 0.4500
Epoch 2/15
400/400 [==============================] - 0s 192us/sample - loss: 1.0977 - accuracy: 0.4900 - val_loss: 1.1052 - val_accuracy: 0.4100
Epoch 3/15
400/400 [==============================] - 0s 158us/sample - loss: 1.0424 - accuracy: 0.5000 - val_loss: 1.0691 - val_accuracy: 0.4500
Epoch 4/15
400/400 [==============================] - 0s 89us/sample - loss: 0.9863 - accuracy: 0.5575 - val_loss: 1.0402 - val_accuracy: 0.5100
Epoch 5/15
400/400 [==============================] - 0s 133us/sample - loss: 0.9533 - accuracy: 0.5700 - val_loss: 1.0342 - val_accuracy: 0.5100
Epoch 6/15
400/400 [==============================] - 0s 101us/sample - loss: 0.9368 - accuracy: 0.5825 - val_loss: 1.0034 - val_accuracy: 0.5700
Epoch 7/15
400/400 [==========================

In [13]:
compareMAandNY

(0.029999971, 0.0)

In [14]:
print(data.keys())

dict_keys(['NC', 'WI', 'NY', 'MD', 'PA', 'CA', 'TX', 'AZ', 'WA', 'IL', 'KY', 'MT', 'IA', 'GA', 'ME', 'VA', 'SC', 'WV', 'FL', 'NH', 'MN', 'NE', 'AR', 'NJ', 'SD', 'NM', 'OK', 'MI', 'VT', 'ID', 'DE', 'MA', 'WY', 'CO', 'IN', 'AL', 'TN', 'HI', 'AK', 'OH', 'RI', 'LA', 'OR', 'KS', 'UT', 'MO', 'DC', 'NV', 'ND', 'MS', 'CT'])


In [15]:
states = ["MA", "NY", "AK", "CO"]
I,J = len(states), len(states)
similarity_list = []

In [9]:
layers_to_transfer = 4
similarity_list.append(np.zeros((2,I,J)))
#We can only transfer at most 4 layers for now
for n in range(1,layers_to_transfer + 1):
    #n of IxJ matrices
    similarity = np.zeros((2,I,J))
    for r in range(I):
        fromState = states[r]
        for c in range(J):
            toState = states[c]
            print("Processing", fromState, toState, "layers transferred:", n)
            if fromState != toState:
                transferLearner = TransferLearn(fromStateData = data[fromState], toStateData = data[toState], layers_to_transfer=n,verbose=0)
                frozen_diff, unfrozen_diff = transferLearner.compare()
            else:
                frozen_diff, unfrozen_diff = 0, 0
            similarity[0][r][c] = frozen_diff
            similarity[1][r][c] = unfrozen_diff
    print(similarity)
    similarity_list.append(similarity)
    

Processing MA MA layers transferred: 1
Processing MA CA layers transferred: 1
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing MA NY layers transferred: 1
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing CA MA layers transferred: 1
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: Tru

*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing NY CA layers transferred: 4
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing NY NY layers transferred: 4
[[[ 0.00000000e+00 -8.01265240e-04  4.60267067e-03]
  [-6.53188229e-02  0.00000000e+00 -3.08471918e-03]
  [-1.86625123e-02  9.93609428e-04  0.00000000e+00]]

 [[ 0.00000000e+00  3.23730707e-03 -8.03011656e-03]
  [-2.48833895e-02  0.00000000e+00 -2.00748444e-03]
  [ 1.71073079e-02  9.61422920e-05  0.00000000e+00]]]


In [11]:
for i in range(len(similarity_list)):
    print("Transferred",i,"layers:")
    print(similarity_list[i])

Transferred 0 layers:
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
Transferred 1 layers:
[[[ 0.          0.00743616 -0.00063652]
  [ 0.01244169  0.         -0.0251677 ]
  [-0.01088649  0.00227571  0.        ]]

 [[ 0.          0.00278854  0.00088137]
  [ 0.01244169  0.         -0.02536356]
  [-0.01088649 -0.00192314  0.        ]]]
Transferred 2 layers:
[[[ 0.          0.0048399   0.00665915]
  [-0.0279938   0.         -0.00225234]
  [ 0.02021772 -0.01631463  0.        ]]

 [[ 0.          0.00608993  0.00014687]
  [ 0.01866251  0.          0.00053859]
  [ 0.0155521  -0.00842977  0.        ]]]
Transferred 3 layers:
[[[ 0.         -0.00522453  0.0012241 ]
  [ 0.06220835  0.          0.00254619]
  [-0.00622082 -0.00480783  0.        ]]

 [[ 0.          0.00012821  0.00014687]
  [ 0.10575426  0.         -0.00509226]
  [-0.00933129 -0.01230806  0.        ]]]
Transferred 4 layers:
[[[ 0.00000000e+00 -8.01265240e-04  4.60267067e-03]
  [-6.53188229e-02  0.00

In [16]:
layers_to_transfer = 4
similarity_list.append(np.zeros((2,I,J)))
#We can only transfer at most 4 layers for now
for n in range(1,layers_to_transfer + 1):
    #n of IxJ matrices
    similarity = np.zeros((2,I,J))
    for r in range(I):
        fromState = states[r]
        transferLearner = TransferLearn(fromState = fromState, toState = None, layers_to_transfer=n,verbose=0)
        transferLearner.initFromStateData()
        transferLearner.getFromModelWeights()
        for c in range(J):
            toState = states[c]
            print("Processing", fromState, toState, "layers transferred:", n)
            if fromState != toState:
                transferLearner.toState = toState
                transferLearner.initToStateData(sample_size=500)
                frozen_diff, unfrozen_diff = transferLearner.compare()
            else:
                frozen_diff, unfrozen_diff = 0, 0
            similarity[0][r][c] = frozen_diff
            similarity[1][r][c] = unfrozen_diff
    print(similarity)
    similarity_list.append(similarity)

Processing MA MA layers transferred: 1
Processing MA NY layers transferred: 1
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing MA AK layers transferred: 1
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing MA CO layers transferred: 1
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: Tru

*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing CO CO layers transferred: 2
[[[ 0.         -0.01999998 -0.00999999  0.02000004]
  [-0.01999998  0.         -0.02000004 -0.06      ]
  [ 0.06999999  0.03000003  0.          0.        ]
  [ 0.06       -0.00999999 -0.22000003  0.        ]]

 [[ 0.         -0.03000003 -0.00999999  0.        ]
  [-0.00999999  0.         -0.02000004 -0.08999997]
  [ 0.04000002  0.03000003  0.         -0.00999999]
  [ 0.04000002 -0.00999999 -0.13999999  0.        ]]]
Processing MA MA layers transferred: 3
Processing MA NY layers transferred: 3
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing MA AK layers transferr

*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing CO NY layers transferred: 4
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing CO AK layers transferred: 4
*****Training Benchmark Model*****
*****Training Benchmark Model Done*****
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: False
*****weights obtained from fromStateData*****
Transferring
*****Transferring done***** Trainable: True
Processing CO CO layers transferred: 4
[[[ 0.         -0.00999999  0.01999998  0.02999997]
  [-0.04000002  0.          0.01999998 -0.06999999]
  [ 0.00999999  0.          0.         -0.01999998]
  [ 0.00999999 

In [17]:
#run 1
for i in range(len(similarity_list)):
    print("Transferred",i,"layers:")
    print(similarity_list[i])

Transferred 0 layers:
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
Transferred 1 layers:
[[[ 0.          0.04999998 -0.01999998  0.00999999]
  [-0.04999995  0.         -0.02000004 -0.05000001]
  [ 0.00999999 -0.04000002  0.         -0.01999998]
  [ 0.02999997  0.         -0.02999997  0.        ]]

 [[ 0.          0.04999998 -0.05000001  0.01999998]
  [-0.00999999  0.         -0.03000003 -0.02999997]
  [-0.01999998 -0.01999998  0.         -0.05000001]
  [ 0.01999998  0.00999999 -0.01999998  0.        ]]]
Transferred 2 layers:
[[[ 0.         -0.01999998 -0.00999999  0.02000004]
  [-0.01999998  0.         -0.02000004 -0.06      ]
  [ 0.06999999  0.03000003  0.          0.        ]
  [ 0.06       -0.00999999 -0.22000003  0.        ]]

 [[ 0.         -0.03000003 -0.00999999  0.        ]
  [-0.00999999  0.         -0.02000004 -0.08999997]
  [ 0.04000002  0.03000003  0.         -0.00999999]
  [ 0.04000002 -